In [2]:
import pandas as pd

model_tfidf_data = pd.read_csv('Data/model_tfidf.csv', index_col=0)
model_bert_data = pd.read_csv('Data/model_bert.csv', index_col=0)

events = pd.read_csv('Data/events.csv')
users = pd.read_csv('Data/users.csv')

users_tfidf = pd.read_csv('Data/users_tfidf.csv').set_index('visitor_token_id')
users_bert = pd.read_csv('Data/users_bert.csv').set_index('visitor_token_id')

In [3]:
from catboost import CatBoostClassifier

tfidf_model = CatBoostClassifier().load_model("Models/tfidf_catboost_v1")
bert_model = CatBoostClassifier().load_model("Models/bert_catboost_v1")

In [4]:
from datetime import datetime
import numpy as np


def get_predictions(token: str):
    current_year = datetime.now().year
    tmp_tfidf = (model_tfidf_data[(model_tfidf_data.index != token) &
                                  (model_tfidf_data['time_year'] == current_year)]
                 .drop(columns=['target', 'time_year'])
                 .drop_duplicates(subset=['ticket_type_id'])
                 .reset_index(drop=True))

    tmp_bert = (model_bert_data[(model_bert_data.index != token) &
                                (model_bert_data['time_year'] == current_year)]
                .drop(columns=['target', 'time_year'])
                .drop_duplicates(subset=['ticket_type_id'])
                .reset_index(drop=True))

    tfidf_tt_ids = tmp_tfidf['ticket_type_id']
    tfidf_bert_ids = tmp_bert['ticket_type_id']

    tmp_tfidf = tmp_tfidf.drop(columns=['ticket_type_id'])
    tmp_bert = tmp_bert.drop(columns=['ticket_type_id'])

    tfidf_user = users_tfidf[users_tfidf.index == token]
    bert_user = users_bert[users_bert.index == token]

    tmp_tfidf[tfidf_user.columns] = tfidf_user.to_numpy().repeat(len(tmp_tfidf), axis=0)
    tmp_bert[bert_user.columns] = bert_user.to_numpy().repeat(len(tmp_bert), axis=0)

    tfidf_result = tfidf_model.predict_proba(tmp_tfidf)
    bert_result = bert_model.predict_proba(tmp_bert)

    top_5_tfidf = (tfidf_tt_ids[np.argsort(tfidf_result[:, 1])[-5:]]
                   .rename('tfidf_res')
                   .reset_index(drop=True))

    top_5_bert = (tfidf_bert_ids[np.argsort(bert_result[:, 1])[-5:]]
                  .rename('bert_res')
                  .reset_index(drop=True))

    return pd.concat([top_5_tfidf, top_5_bert], axis=1)


get_predictions(token='0000b08340cc86c7a61cc3251fbf2544c6472dda')

,tfidf_res,bert_res
0,1124,1022
1,922,1090
2,917,1134
3,923,1079
4,1092,915


In [5]:
users

,visitor_token_id,job,params
0,0000b08340cc86c7a61cc3251fbf2544c6472dda,бариста наставник,NaN
1,0004214be6142004e5dfae0c869f7648c33e3640,менеджер по работе с ключевыми клиентами,NaN
2,00099e4722876bce6828a4d083f59d9bc80284d1,NaN,ресторан кафе продукты питания и напитки
3,000aa3c29eee38748aa67148ec80f8754f27017f,NaN,кулинарная студия
4,000cf2d412eb10cae69abe61b606e8be3063d9ba,NaN,ресторан кафе ресторан кафе
...,...,...,...
38644,fff47c137b2ba6a66e4670fe3701f3088682b86e,су шеф кондитер,городской отель изучение рынка знакомство с но...
38645,fff5ecf120366e7a9982e7e01666f84280478336,NaN,NaN
38646,fff6302be5388f9160e86cb11a6be2cf487a1c5b,founder,кофейня общение с коллегами установление новых...
38647,fff676d8522543716287e474ed6eebe50db360ab,генеральный директор,изучение рынка знакомство с новинками получени...


In [8]:
tfidf_predictions = get_predictions(token='fff47c137b2ba6a66e4670fe3701f3088682b86e').iloc[:, 0]
events[events['ticket_type_id'].isin(tfidf_predictions)].set_index('ticket_type_id')

,name,description,time_start,price,type_name,theme
ticket_type_id,,,,,,
1176,гостиничная индустрия тренды в подходах развит...,устойчивое развитие технологические инновации ...,2024-10-22 12:40:00+00:00,0.0,экспертная сессия,отель
927,код к мотивации сотрудников или почему достигн...,достижение высокого уровня сервиса в отельном ...,2024-10-22 10:00:00+00:00,2700.0,мастер класс,отель
1169,искусственный интеллект на службе у отельеров,быстро развивающиеся технологии искусственного...,2024-10-21 12:50:00+00:00,0.0,экспертная сессия,отель
1150,облачные pms современные запросы от отеля,NaN,2024-10-21 10:50:00+00:00,0.0,экспертная сессия,отель
1127,стратегическая сессия итоги сезона и перспективы,гостиницы итоги сезона ключевые операционные п...,2024-10-21 12:15:00+00:00,0.0,панельная дискуссия,отель


In [9]:
bert_predictions = get_predictions(token='fff47c137b2ba6a66e4670fe3701f3088682b86e').iloc[:, 1]
events[events['ticket_type_id'].isin(bert_predictions)].set_index('ticket_type_id')

,name,description,time_start,price,type_name,theme
ticket_type_id,,,,,,
929,как предотвратить выгорание сотрудников первой...,мероприятие посвященное важной теме предотвращ...,2024-10-23 12:15:00+00:00,2700.0,семинар,отель
985,конференция успешный глэмпинг без ошибок учимс...,соорганизатор ассоциация глэмпингов россии сес...,2024-10-23 07:10:00+00:00,9500.0,конференция,отель
1151,автоматизация и уменьшение человеческого факто...,модератор эдуард демидов спикер анна крампец н...,2024-10-23 07:30:00+00:00,0.0,экспертная сессия,отель
932,увеличение прибыли и загрузки объекта с помощь...,на лекции практикуме спикеры подробно обсудят ...,2024-10-22 10:00:00+00:00,2700.0,семинар практикум,отель
938,финансовый эффект от реновации объекта,мероприятие посвященное реновации объектов в о...,2024-10-22 07:15:00+00:00,2700.0,семинар,отель


Следующая задача заключается в разбиении текстов на более мелкие тематики - Пицца, Алкоголь, Кондитерка